In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()

In [ ]:
lonMin = (float)(config['TRAJ_PRED_LSTM']['LON_MIN'])
lonMax = (float)(config['TRAJ_PRED_LSTM']['LON_MAX'])

latMin = (float)(config['TRAJ_PRED_LSTM']['LAT_MIN'])
latMax = (float)(config['TRAJ_PRED_LSTM']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TRAJ_PRED_LSTM']['INCR_STEP'])
incrRes = (int)(config['TRAJ_PRED_LSTM']['INCR_RES'])

sourceDir = config['TRAJ_PRED_LSTM']['SOURCE_DIR']
print(sourceDir)

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
#first read the data frame
#convert LON and LAT into cooresponding 
# def get_index_from_lon_lat(lat,lon):
def get_index_from_lon_lat(latLonRow):
    retVal = -1
    lat = latLonRow['LAT']
    lon = latLonRow['LON']
    for boundary in boundaryArray: 
        if(lon >= boundary[0]) and (lon < boundary[1]) \
            and (lat >= boundary[2]) and (lat < boundary[3]):
            retVal = boundary[4]
            break 
    return retVal

#will convert dataframe into sequence of numbers
#which can be used to make data for LSTM
def convert_traj_df_to_state_sequence(num):
    #read the dataframe
    sorceFile = sourceDir + str(num) + '.csv'
    sourceDF,_ = aISDM.load_data_from_csv(sorceFile)
    #conver every LON and LAT to sequence of numbers
    ret = sourceDF.apply(get_index_from_lon_lat,axis=1)
    return ret.to_numpy()
    
# convert_traj_df_to_state_sequence(0)

In [ ]:
#make list of all such trajectories
#number goes from 0 to 80000
#that much is going to be our training data
trajSeqList = []

#FIXME get number from Config file
for trajNum in range(0,60802):
    trajSeqList.append(convert_traj_df_to_state_sequence(trajNum))

In [ ]:
lessDataCount = 0;
for trajNum in range(0,60802):
    if(len(trajSeqList[trajNum]) <= 2):
        lessDataCount = lessDataCount + 1
        
print(lessDataCount)

In [ ]:
#takes numpy array and returns x and y for
#time series prediction
def convert_seq_to_x_y(seq):
    #first column
    #-2 is is to take care of boundary condition
    #since we are considering 2 time stamps for the input data
    firstCol = seq[:-2].copy()
    firstCol = np.reshape(firstCol,(firstCol.shape[0],1))
#     print(firstCol)
#     print(firstCol.shape)
    #second column shifted by 1 time instances
    secCol = seq[1:-1].copy()
    secCol = np.reshape(secCol,(secCol.shape[0],1))
#     print(secCol)
#     print(secCol.shape)
    #output is shifted by two time instances
    outputLabel = seq[2:].copy()
    outputLabel = np.reshape(outputLabel,(outputLabel.shape[0],1))
#     print(outputLabel)
#     print(outputLabel.shape)
    xData = np.hstack((firstCol,secCol))
#     print(xData.shape)
    return xData, outputLabel
    
# convert_seq_to_x_y(trajSeqList[0])

In [ ]:
#now iterate throgh trajSeqList 
#and keep on stacking them vertically
#to make giant input and output matrix
xData = np.zeros((0,2))
yData = np.zeros((0,1))
print(xData.shape)
print(yData.shape)
for trajNum in range(len(trajSeqList)):
    if(len(trajSeqList[trajNum]) > 2):
        xTemp,yTemp = convert_seq_to_x_y(trajSeqList[trajNum])
        xData = np.vstack((xData,xTemp.copy()))
        yData = np.vstack((yData,yTemp.copy()))
print(xData)
print(yData)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
xToStore = "../Data/M120_50_M119_00_33_90_34_44/1004/LSTMData_17/XData.npy"
yToStore = "../Data/M120_50_M119_00_33_90_34_44/1004/LSTMData_17/YData.npy"
np.save(xToStore, xData)
np.save(yToStore, yData)

In [ ]:
xData = np.load("../Data/M120_50_M119_00_33_90_34_44/1004/LSTMData_17/XData.npy")
yData = np.load("../Data/M120_50_M119_00_33_90_34_44/1004/LSTMData_17/YData.npy")
yData = yData.astype(int)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
def get_lon_lat_from_idx(idx):
    lonMid, latMid = hMUtil.compute_mid_point(boundaryArray[idx][0] \
                                                            ,boundaryArray[idx][1]\
                                                            ,boundaryArray[idx][2]\
                                                            ,boundaryArray[idx][3]\
                                                            )
    return lonMid, latMid

def convert_state_to_lon_lat(sampleData):
    #previous state
    lonPrev, latPrev = get_lon_lat_from_idx((int)(sampleData[0]))
    #current state
    lonCurr, latCurr = get_lon_lat_from_idx((int)(sampleData[1]))
    return np.array([lonPrev, latPrev, lonCurr, latCurr])

In [ ]:
#like vectorised operation
#compute the new xData
xLonLatData = np.apply_along_axis(convert_state_to_lon_lat,1,xData)

In [ ]:
#normalise the data
xLonLatData_0 = (xLonLatData[:,0] - lonMin)/(lonMax - lonMin)
xLonLatData_0 = np.reshape(xLonLatData_0,(xLonLatData_0.shape[0],1))
xLonLatData_1 = (xLonLatData[:,1] - latMin)/(latMax - latMin)
xLonLatData_1 = np.reshape(xLonLatData_1,(xLonLatData_1.shape[0],1))
xLonLatData_2 = (xLonLatData[:,2] - lonMin)/(lonMax - lonMin)
xLonLatData_2 = np.reshape(xLonLatData_2,(xLonLatData_2.shape[0],1))
xLonLatData_3 = (xLonLatData[:,3] - latMin)/(latMax - latMin)
xLonLatData_3 = np.reshape(xLonLatData_3,(xLonLatData_3.shape[0],1))
xLonLatDataNorm = np.hstack((xLonLatData_0,xLonLatData_1,xLonLatData_2,xLonLatData_3))

In [ ]:
print(xLonLatDataNorm.shape)

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from keras.utils import Sequence
from keras.utils import to_categorical

#generator for LON and LAT
class DataGeneratorLonLat(Sequence):
    'Generates data for Keras'
    def __init__(self):
        'Initialization'
        self.sampleIDX = 0
        self.on_epoch_end()
        self.batchSize = 32

    def __len__(self):
        'Denotes the number of batches per epoch'
        return (1900)

    def __getitem__(self, index):
        'Generate one batch of data'
        batchFeatures = xLonLatDataNorm[self.sampleIDX:(self.sampleIDX+self.batchSize),:].copy()
        batchFeatures = np.reshape(batchFeatures,(self.batchSize, 2, 2))
        batchLabels = to_categorical(yData[self.sampleIDX:(self.sampleIDX+self.batchSize),:].copy(),num_classes = totalStates)
        self.sampleIDX = self.sampleIDX + 32
        return batchFeatures, batchLabels

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        print("Epoch End")
        self.sampleIDX = 0

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences= True, input_shape=(2,2)))
model.add(LSTM(units=50))
model.add(Dense(units=totalStates, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
training_generator = DataGeneratorLonLat()

In [ ]:
model.fit_generator(training_generator, epochs=500, verbose = 2)

In [ ]:
#model.save("../Data/M120_50_M119_00_33_90_34_44/1004/Models_17/Model_2000.h5")

In [ ]:
from keras.models import load_model
model = load_model("../Data/M120_50_M119_00_33_90_34_44/1004/Models_17/Model_2000.h5")

In [ ]:
sampleID  = 60
#lets try to predict some values
print(xData[sampleID])
print(yData[sampleID])
print(xLonLatDataNorm[sampleID])

xTestModel = xLonLatDataNorm[sampleID].copy()
xTestModel = np.reshape(xTestModel,(1,2,2))

yHatProb = model.predict(xTestModel)
print(yHatProb.shape)
yHatProb = yHatProb.T
yHatProb = yHatProb.flatten()
print(yHatProb.shape)
maxLoc = yHatProb.argsort()[-3:][::-1]
print(maxLoc)
print(boundaryArray[maxLoc[0]])
print(boundaryArray[2672])
print(yHatProb[maxLoc[0]])
print(yHatProb[maxLoc[1]])
print(yHatProb[maxLoc[2]])

In [ ]:
print(np.sum(yHatProb))

In [ ]:
#will take four arguments
#[[lonPrev, latPrev, lonCurr, latCurr]] shape is 1x4 or nx4
#can be vectorised also
def normalize_lon_lat(arr):
    #subtract the minimum 
    #and divide by range
    ret02 = (arr[:,[0,2]] - lonMin)/(lonMax - lonMin)
    ret13 = (arr[:,[1,3]] - latMin)/(latMax - latMin)
    ret0213 = np.hstack((ret02, ret13))
    ret0213[:,[0,1,2,3]] = ret0213[:,[0,2,1,3]]
    return ret0213


def test_normalize_lon_lat():
    dummyArr = np.array([-119.27, 34.03, -119.26, 34.04])
    print(dummyArr.shape)
    print(dummyArr)
    dummyArr = np.reshape(dummyArr , (dummyArr.shape[0],1))
    print(dummyArr.shape)
    print(dummyArr)
    dummyArr = dummyArr.T
    print(dummyArr.shape)
    print(dummyArr)
    print(normalize_lon_lat(dummyArr))
    
normalize_lon_lat(np.array([[-119.27, 34.03, -119.26, 34.04] \
                            ,[-119.26, 34.04, -119.27, 34.03]]))
test_normalize_lon_lat()

In [ ]:
#function computes top1 and top3 error 
#for 30 minutes prediction
#takes 3 consecutive trajectory states
#prev current and next 
#in list formate or flatten numpy array
#[prev, current, next]
def compute_30_min_error(trajState):
    #convert the first data to lon and lat
    initalTraj = convert_state_to_lon_lat(np.array([trajState[0],trajState[1]]))
#     print(trajState[0])
#     print(trajState[1])
    #now use it to predict the model
    #for 30 minutes prediction
#     print(initalTraj)
#     print(len(initalTraj.shape))
    #check for the shape
    #to put into sample x feature formate
#     print(initalTraj)
    if(len(initalTraj.shape) < 2):
        initalTraj = np.reshape(initalTraj , (1,initalTraj.shape[0]))

    mSample = initalTraj.shape[0]

    #first normalise the data
    initalTrajNorm = normalize_lon_lat(initalTraj)
#     print(initalTrajNorm)
    #use the normalised to predict
    #reshape input as samples x features x timestamps
    initalTrajNorm = np.reshape(initalTrajNorm,(mSample,2,2))
#     print(initalTrajNorm)
    pred30 = model.predict(initalTrajNorm)
    #Lets take top 3
    pred30Top3 = pred30.argsort()[-3:][::-1]
#     print(pred30Top3.shape)
    #now compute distance with top31,top32 and top33
    top31 = pred30Top3[0,0]
    top32 = pred30Top3[0,1]
    top33 = pred30Top3[0,2]
#     print(top31)
#     print(top32)
#     print(top33)
    top31LonLat = get_lon_lat_from_idx(top31)
    top32LonLat =  get_lon_lat_from_idx(top32)
    top33LonLat =  get_lon_lat_from_idx(top33)
    #get LON and LAT for true label
    trueLonLat = get_lon_lat_from_idx(trajState[2])
#     print(trueLonLat)
#     print(top31LonLat)
#     print(top32LonLat)
#     print(top33LonLat)
    
    top31Err = gC.compute_distance(trueLonLat[0], trueLonLat[1] \
                                      ,top31LonLat[0], top31LonLat[1])

    top32Err = gC.compute_distance(trueLonLat[0], trueLonLat[1] \
                                      ,top32LonLat[0], top32LonLat[1])

    top33Err = gC.compute_distance(trueLonLat[0], trueLonLat[1] \
                                      ,top33LonLat[0], top33LonLat[1])

    print(top31Err, top32Err, top33Err)
    return top31Err, min(top31Err, top32Err, top33Err)

In [ ]:
top1Err30Min = []
top3Err30Min = []
for tarj in range(60802):
    trajState = convert_traj_df_to_state_sequence(tarj)
    if(len(trajState) > 2):
        tempTop1Err, tempTop3Err = compute_30_min_error(trajState)
        top1Err30Min.append(tempTop1Err)
        top3Err30Min.append(tempTop3Err)

In [ ]:
print(len(top1Err30Min))
print(len(top3Err30Min))

In [ ]:
top1Err30MinArr = np.array(top1Err30Min)
top3Err30MinArr = np.array(top3Err30Min)

In [ ]:
def format_func(value, tick_number):
    tempTick = (value*30) + 30
    ret = "%d"%(tempTick)
    return ret

In [ ]:
plt.hist(top1Err30Min)

In [ ]:
print(np.mean(top1Err30MinArr))
print(np.mean(top3Err30MinArr))